In [ ]:
using GraphPlot: gplot, circular_layout

In [ ]:
include("data_graph.jl")

In [ ]:
# Actually this works! Got it from:
# https://discourse.julialang.org/t/re-importing-modules-with-changes-in-the-code/9192/4
Revise.track("data_graph.jl")

In [ ]:
using .MyDataGraph

In [ ]:
gen_graphs_hard

In [ ]:
graphs = gen_graphs_hard(DataSpec(d=10, k=1, gtype=:ER,
                                      noise=:Gaussian))[1:11]

In [ ]:
g = graphs[1]

In [ ]:
gplot(g, nodelabel=1:nv(g), layout=circular_layout)

In [ ]:
myplot(g)

# General Setting and Importing

In [ ]:
ENV["COLUMNS"] = 200

In [ ]:
import CSV

In [ ]:
using DataFrames

In [ ]:
include("data_graph.jl")

# Load a model

In [ ]:
# directly run the model
include("exp.jl")

In [ ]:
# load the trained model
fname = "saved_models/EQ-d=20_k=1_gtype=SF_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson"
@load fname model

In [ ]:
@load "saved_models/EQ-d=10_k=1_gtype=ER_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "back/back-0907/CORCOV/EQ-d=10_k=1_gtype=SF_noise=Gaussian_mat=COR_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/ensK-2020-09-08T10:58:41.247-ensemble/step-10000.bson" model

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
@load "saved_models/ensemEQ-CH2-1,2,4-2020-10-11T11:29:01.183-ensemble/step-100000.bson" model

In [ ]:
@load "saved_models/ensemEQ-CH3-1:2:20-2020-10-14T19:02:47.266-ensemble/step-80987.bson" model

In [ ]:
fname = "saved_models/ensemEQ-CH3-1:2:20-2020-10-14T19:02:47.266-ensemble/step-84362.bson"
@load fname model

In [ ]:
@load "saved_models/ensemEQ-CH3-d=20-1:2:20-ensemble/step-100000.bson" model

# Evaluate functions

In [ ]:
function evaluate_real_acyclic(out, greal, dfnames)
    Wout = threshold(σ.(out), 0.3, false)
    # find the order of the index, or, sort the indexes
    sort(Wout, dims=1)
    edgeidx = findall((x)->x>0, Wout)
    sorted_idx = edgeidx[sortperm(Wout[edgeidx], rev=true)]
    # add sorted idx
    g = MetaDiGraph(size(out, 1))
    for idx in sorted_idx
        add_edge!(g, idx[1], idx[2])
        if is_cyclic(g)
            rem_edge!(g, idx[1], idx[2])
        end
    end
    p1 = myplot(g, dfnames)
    p0 = myplot(greal, dfnames)
    adj1 = adjacency_matrix(g)
    adj0 = adjacency_matrix(greal)
    # predicted edge, true edge, SHD
    predicted_edge = ne(g)
    @show predicted_edge
    correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
    @show correct_edge
    reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
    @show reversed_edge
    @show length(edges(greal))

    # metrics
    ytrue = Matrix(gen_weights(greal))
    metrics = sup_graph_metrics(adj1, ytrue)
    return p0, p1, metrics
end

In [ ]:
function evaluate_real(out, greal, dfnames)
    Wout = threshold(σ.(out), 0.3, true)
    # predicted edge, true edge, SHD
    predicted_edge = ne(DiGraph(Wout))
    @show predicted_edge
    correct_edge = sum(Wout[Wout .== 1] .== adjacency_matrix(greal)[Wout .== 1])
    @show correct_edge
    @show length(edges(greal))

    # metrics
    ytrue = Matrix(gen_weights(greal))
    metrics = sup_graph_metrics(Wout, ytrue)
    p1 = myplot(DiGraph(Wout), dfnames)
    p0 = myplot(greal, dfnames)
#     plot(p0, p1)
    return p0, p1, metrics
end

## Evaluate the model

In [ ]:
include("exp.jl")

### prepare the data

In [ ]:
ch3X = getch3(X)

### do the inference

In [ ]:
out = inf_one(model, medcovX)

In [ ]:
out = inf_one(model, corX)

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
out = inf_one(model, ch2X)

In [ ]:
out = inf_one(model, ch3X)

### examine the results

In [ ]:
Wout = threshold(σ.(out), 0.5, true)

In [ ]:
myplot(DiGraph(Wout), names(df))

In [ ]:
myplot(greal)

In [ ]:
myplot(syntrenG, names(df))

In [ ]:
edges(syntrenG)

In [ ]:
# predicted edge, true edge, SHD
predicted_edge = ne(DiGraph(Wout))
@show predicted_edge
correct_edge = sum(Wout[Wout .== 1] .== adjacency_matrix(syntrenG)[Wout .== 1])
@show correct_edge

# metrics
ytrue = Matrix(gen_weights(syntrenG))
sup_graph_metrics(Wout, ytrue)

In [ ]:
# TODO calculate #reverse direction edges
sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])

In [ ]:
sum(Wout'[Wout' .== 1] .== adjacency_matrix(SachsG)[Wout' .== 1])

In [ ]:
# TODO implement the recursive add procedure to remove cycles
# Or, just construct the graph, and keep removing until it is a DAG
is_cyclic(DiGraph(Wout))

In [ ]:
Wout

In [ ]:
# Or just implement the procedure
out

### using evaluation functions

In [ ]:
using Plots

In [ ]:
p0, p1, metrics = evaluate_real(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

### evaluation function 0

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

### evaluation function 1

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

### evaluation function 2

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

# Sachs 2005

http://www.sciencemag.org/content/suppl/2005/04/21/308.5721.523.DC1/Sachs.SOM.Datasets.zip

The paper: Causal Protein-Signaling Networks Derived from Multiparameter Single-Cell Data 

In [ ]:
url = "http://www.sciencemag.org/content/suppl/2005/04/21/308.5721.523.DC1/Sachs.SOM.Datasets.zip"
download(url, "Sachs.zip")

In [ ]:
unzip

In [ ]:
# TODO use a pre-defined name "Sachs"
run(`unzip Sachs.zip`)

In [ ]:
# It should be 11 column
df = CSV.read("Sachs/1.cd3cd28.csv");

In [ ]:
X = convert(Matrix, df)

In [ ]:
greal = Sachs_ground_truth()

In [ ]:
myplot(greal)

In [ ]:
edges(greal)

In [ ]:
nv(greal)

## Evaluate

In [ ]:
fname = "saved_models/ensemEQ-CH3-1:2:20-2020-10-14T19:02:47.266-ensemble/step-84362.bson"
@load fname model

In [ ]:
ch3X = getch3(X);

In [ ]:
out = inf_one(model, ch3X);

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

## Compare SachsX and X

In [ ]:
cov(SachsX)

In [ ]:
cov(X)

In [ ]:
cov(SachsX) ./ maximum(var(SachsX, dims=1))

In [ ]:
cov(X) ./ maximum(var(X, dims=1))

In [ ]:
cov(X) ./ median(var(X, dims=1))

In [ ]:
cov(SachsX) ./ median(var(SachsX, dims=1))

In [ ]:
cor(X)

In [ ]:
cor(SachsX)

In [ ]:
var(X, dims=1)

In [ ]:
var(SachsX, dims=1)

In [ ]:
# The hyper-parameters for generating synthetic data
# - C scale (0.5, 0.5+k)
# - mu/sigma of errors

In [ ]:
W10 = gen_weights(g, ()->((rand() * 10.5 + 0.5) * rand([1,-1])))

In [ ]:
X10 = gen_data2(W10, :Gaussian, 1000)

In [ ]:
cov(X10)

In [ ]:
var(X10, dims=1)

# bnlearn datasets

- CRAN package link: https://cran.r-project.org/src/contrib/bnlearn_4.6.1.tar.gz
  - I could download the tar ball, uncompress it, and copy the data/ directory out
- Or I could download the links to the .rda files individually. I'm trying this first.
  - https://www.bnlearn.com/documentation/networks/gaussian.test.rda
  - https://www.bnlearn.com/bnrepository/ecoli70/ecoli70.rda
  - https://www.bnlearn.com/bnrepository/magic-niab/magic-niab.rda
  - https://www.bnlearn.com/bnrepository/magic-irri/magic-irri.rda
  - https://www.bnlearn.com/bnrepository/sangiovese/sangiovese.rda
  - https://www.bnlearn.com/bnrepository/mehra/mehra-complete.rda

In [ ]:
mkdir("data/bnlearn")

In [ ]:
# set proxy, neoproxy
ENV["http_proxy"] = "http://172.18.0.1:8889"
ENV["https_proxy"] = "http://172.18.0.1:8889"

In [ ]:
urls = 
  [ "https://www.bnlearn.com/documentation/networks/gaussian.test.rda",
   "https://www.bnlearn.com/bnrepository/ecoli70/ecoli70.rda",
  "https://www.bnlearn.com/bnrepository/magic-niab/magic-niab.rda",
  "https://www.bnlearn.com/bnrepository/magic-irri/magic-irri.rda",
  "https://www.bnlearn.com/bnrepository/sangiovese/sangiovese.rda",
  "https://www.bnlearn.com/bnrepository/mehra/mehra-complete.rda"]

In [ ]:
for url in urls
    fname = joinpath("data/bnlearn/", split(url, "/")[end])
    if !isfile(fname)
        download(url, fname)
    end
end

In [ ]:
import RData

In [ ]:
objs = RData.load("data-back/bnlearn/gaussian.test.rda")

In [ ]:
objs["bn"]

In [ ]:
keys(objs)

In [ ]:
keys(objs["bn"]["A"])

In [ ]:
objs["bn"]["A"]["fitted.values"]

## real data for bnlearn

In [ ]:
df = CSV.read("data-back/gaussian.dat", delim=" ");

In [ ]:
X = convert(Matrix, df)

In [ ]:
var(X, dims=1)

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(inf_one(model, getch3(X)), greal, ["A", "B", "C", "D", "E", "F", "G"]);

In [ ]:
p0

In [ ]:
p1

In [ ]:
function bnlearn_ground_truth()
    greal = named_graph([:A, :B, :C, :D, :E, :F, :G])
    named_graph_add_edge!(greal, :B, :C)
    named_graph_add_edge!(greal, :A, :C)
    named_graph_add_edge!(greal, :B, :D)
    named_graph_add_edge!(greal, :D, :F)
    named_graph_add_edge!(greal, :A, :F)
    named_graph_add_edge!(greal, :G, :F)
    named_graph_add_edge!(greal, :E, :F)
    greal
end

In [ ]:
greal = bnlearn_ground_truth()

In [ ]:
bnp0 = myplot(greal, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
bnp1 = myplot(g, ["A", "B", "C", "D", "E", "F", "G"])

# syntren

- http://bioinformatics.intec.ugent.be/kmarchal/SynTReN/

In [ ]:
# normalized data
# fname = "data/syntren/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_maxExpr1_dataset.txt"

In [ ]:
# unnormalized data
fname = "data/syntren/s1/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_unnormalized_dataset.txt"

In [ ]:
# hop0
fname = "data/syntren/hop0/nn20_nbgr0_hop0.0_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_unnormalized_dataset.txt"

In [ ]:
df = read_syntren(fname);

In [ ]:
X = convert(Matrix, df);

In [ ]:
greal = syntren_ground_truth("data/syntren/s1/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_network.sif",
    names(df))

In [ ]:
# hop0
fname = "data/syntren/hop0/nn20_nbgr0_hop0.0_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_network.sif"
greal = syntren_ground_truth(fname, names(df))

In [ ]:
myplot(syntrenG, names(df))

In [ ]:
# DEBUG make sure the order of names for data and ground truth graph match
# myplot(syntrenG)

## Evaluate

In [ ]:
@load "saved_models/ensemEQ-CH3-d=20-1:2:20-ensemble/step-100000.bson" model

In [ ]:
ch3X = getch3(X);

In [ ]:
out = inf_one(model, ch3X);

In [ ]:
p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));

In [ ]:
metrics

In [ ]:
p0

In [ ]:
p1

## Test all 10 samples

In [ ]:
for seed in 1:10
    @info "processing seed $seed .."
    fname = "data/syntren/s$seed" * "/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_unnormalized_dataset.txt"
    df = read_syntren(fname);
    X = convert(Matrix, df);
    greal = syntren_ground_truth("data/syntren/s$seed/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_network.sif",
    names(df))
    ch3X = getch3(X)
    @info "Performing inference .."
    out = inf_one(model, ch3X)
    @info "Evaluating .."
    p0, p1, metrics = evaluate_real_acyclic(out, greal, names(df));
    @show metrics
end

# Other random staff

## Seeding and deterministic graph generation

- set seeding before generation of each size/type graph
- compare if they are same, across machines
- give it a special filename with seeds used

In [ ]:
include("data_graph.jl")

In [ ]:
spec = DataSpec(d=10, k=1, gtype=:SF, noise=:Gaussian, mechanism=:Linear, mat=:COR)

In [ ]:
ds, test_ds = load_sup_ds(spec, spec.bsize)

In [ ]:
x, y = next_batch!(ds)

In [ ]:
@show size(x)
@show size(y)

In [ ]:
# generating graphs
graphs = gen_graphs_hard(spec)

In [ ]:
size(graphs)

In [ ]:
graphs[1]

In [ ]:
myplot(graphs[1])

In [ ]:
# it is different for different runs
graphs = gen_graphs_hard(spec)
@show graphs[1]
myplot(graphs[1])

In [ ]:
# set seeding
Random.seed!(1234)
# it is different for different runs
graphs = gen_graphs_hard(spec)
@show graphs[1]
myplot(graphs[1])

## Generate covariance / max_varinclude("data_graph.jl")

In [ ]:
include("data_graph.jl")

In [ ]:
g = gen_ER_dag(10)

In [ ]:
myplot(g)

In [ ]:
W = gen_weights(g, ()->((rand() * 1.5 + 0.5) * rand([1,-1])))

In [ ]:
X = gen_data2(W, :Gaussian, 1000)

In [ ]:
# the variance
cov(X)

In [ ]:
cor(X)

In [ ]:
# the variance?
var(X, dims=1)

In [ ]:
maximum(var(X, dims=1))

In [ ]:
maximum(cov(X))

In [ ]:
cov(X) ./ maximum(var(X, dims=1))

### ch2

In [ ]:
using LinearAlgebra: Diagonal

In [ ]:
Diagonal(var(X, dims=1))

In [ ]:
reshape(var(X, dims=1), 10)

In [ ]:
dropdims(var(X, dims=1), dims=1)

In [ ]:
[1,2,3]

In [ ]:
Diagonal([1,2,3])

In [ ]:
ch2 = Diagonal(dropdims(var(X, dims=1), dims=1))

In [ ]:
ch2[1,1]

In [ ]:
ch2[1,2]

In [ ]:
cat(cor(X), ch2, dims=3)

## Test load_sup_ds

In [ ]:
using Dates: now

In [ ]:
include("data_graph.jl")

In [ ]:
# test the new data generation
spec = DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:medCOV)

In [ ]:
# test the new data generation
spec = DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)

In [ ]:
spec = DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:CH3)

In [ ]:
graphs = gen_graphs_hard(spec)

In [ ]:
myplot(graphs[1])

In [ ]:
mats = Matrix.(adjacency_matrix.(graphs[1:15]))

In [ ]:
mats[1]

In [ ]:
cat(mats..., dims=3)

In [ ]:
VectorOfArray(mats)

In [ ]:
ds, test_ds = load_sup_ds(spec, 16, suffix="-$(now())")

In [ ]:
x, y = next_batch!(ds)

In [ ]:
x

### CH3

In [ ]:
spec = DataSpec(d=10, k=20, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:CH3)

In [ ]:
ds, test_ds = load_sup_ds(spec, suffix="-$(now())")

In [ ]:
x, y = next_batch!(ds);

In [ ]:
x

## generate k to match test data

In [ ]:
include("data_graph.jl")

In [ ]:
include("exp.jl")

In [ ]:
using Plots

In [ ]:
using LinearAlgebra

In [ ]:
ENV["COLUMNS"] = 200

In [ ]:
specs = map(1:20) do k
    DataSpec(d=20, k=k, gtype=:ER,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)
end

In [ ]:
# I want to plot for each spec
dses = map(specs) do spec
    ds, _ = spec2ds(spec)
    ds
end

In [ ]:
function plot_one(ds)
    x, y = next_batch!(ds)
    # 1. the variance
    v = map(1:size(x,4)) do i
        var(diag(x[:,:,1,i]))
    end
    # remove top 10%
    v = sort(v)[1:Int(round(size(v,1)*0.8))]
    plot(v)
end

In [ ]:
function plot_all(dses)
    "Plot all into one graph."
    vs = map(dses) do ds
        x, y = next_batch!(ds)
        v = map(1:size(x,4)) do i
            mean(diag(x[:,:,1,i]))
#             maximum(LinearAlgebra.eigen(x[:,:,1,i]).values)
        end
        v = sort(v)[1:Int(round(size(v,1)*1))]
    end
    plot(log10.(cat(vs..., dims=1)))
end

In [ ]:
plot_all(dses)

In [ ]:
plot_all(dses)

### the syntren

In [ ]:
# and this is the Syntren one
var(X, dims=1)

In [ ]:
var(var(X, dims=1)) |> log10

In [ ]:
mean(var(X, dims=1)) |> log10

#### Using different seeds

In [ ]:
# unnormalized data
fname = "data/syntren/s1/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_unnormalized_dataset.txt"

In [ ]:
Xs = map(1:10) do seed
    fname = "data/syntren/s$seed" * "/nn20_nbgr0_hop0.3_bionoise0.1_expnoise0.1_corrnoise0.1_neighAdd_unnormalized_dataset.txt"
    df = read_syntren(fname);
    X = convert(Matrix, df);
    end;

In [ ]:
# This seems to be very stable
for X in Xs
    # about 13
    @show var(var(X, dims=1)) |> log10
    # about 6.3
    @show mean(var(X, dims=1)) |> log10
end

### The Sachs

In [ ]:
var(SachsX, dims=1)

In [ ]:
var(var(SachsX, dims=1)) |> log10

In [ ]:
mean(var(SachsX, dims=1)) |> log10

### Other

In [ ]:
map(dses) do ds
    plot_one(ds)
end

In [ ]:
plot(map(dses) do ds
        plot_one(ds) end...)

In [ ]:
plot_one(dses[1])

In [ ]:
plot_one(dses[2])

In [ ]:
plot_one(dses[3])

In [ ]:
plot_one(dses[4])

In [ ]:
plot_one(dses[5])

In [ ]:
plot_one(dses[6])

In [ ]:
plot_one(dses[7])

In [ ]:
plot_one(dses[8])

In [ ]:
# TODO calculate average for the spec